In [1]:
import os
import re
import time
import random
import string
import pandas as pd
import numpy as np
import gensim as gs
import tensorflow as tf
import collections as col
from pathlib import Path

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Functions

In [2]:
def text_preprocessing(text):
    return re.findall(f"[{string.punctuation}]|[\w]+|[\s\t\r\n]", text.lower())

def create_dictionary(file):
    return gs.corpora.Dictionary([text_preprocessing(line) for line in open(file, "r").readlines()])

In [3]:
vocab = create_dictionary("./stories.txt")

In [4]:
def vocab_encode(text):
    return vocab.doc2idx(text_preprocessing(text))

def vocab_decode(array):
    return ' '.join([vocab.get(idx) for idx in array])

def read_data(filename, window, overlap):
    lines = [line.strip() for line in open(filename, "r").readlines()]

    while True:
        random.shuffle(lines)

        for line in lines:
            words = vocab_encode(line)
            
            for start in range(0, len(words) - window, overlap):
                chunk = words[start: start + window]

                yield chunk

def read_batch(stream, batch_size):
    batch = []
    for element in stream:
        batch.append(element)
        if len(batch) == batch_size:
            yield batch
            batch = []
    yield batch

## Hyperparameter

In [5]:
hidden_sizes = [128, 256]
batch_size = 64
learning_rate = 0.01
skip = 50
num_steps = 50 # for RNN unroled
len_generated = 300

## Model

In [6]:
class RNN(object):
    def __init__(self, model):
        self.model = model
        self.path = f"{self.model}.txt"

        self.seq = tf.placeholder(tf.int32, [None, None], name='seq')
        self.temp = tf.constant(1.5, name='temp')
        self.gstep = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    def create_rnn(self, seq):
        layers = [tf.nn.rnn_cell.GRUCell(size) for size in hidden_sizes]
        cells = tf.nn.rnn_cell.MultiRNNCell(layers)
        
        batch = tf.shape(seq)[0]
        zero_states = cells.zero_state(batch, dtype=tf.float32)
        
        self.in_state = tuple([
            tf.placeholder_with_default(state, [None, state.shape[1]])
            for state in zero_states
        ])

        # this line to calculate the real length of seq
        # all seq are padded to be of the same length, which is num_steps
        length = tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
        self.output, self.out_state = tf.nn.dynamic_rnn(cells, seq, length, self.in_state)

    def create_model(self):
        seq = tf.one_hot(self.seq, len(vocab))

        self.create_rnn(seq)

        self.logits = tf.layers.dense(self.output, len(vocab))
        
        loss = tf.nn.softmax_cross_entropy_with_logits(
            logits=self.logits[:, :-1], labels=seq[:, 1:]
        )

        self.loss = tf.reduce_sum(loss)

        # sample the next character from Maxwell-Boltzmann Distribution 
        # with temperature temp. It works equally well without tf.exp
        self.sample = tf.multinomial(tf.exp(self.logits[:, -1] / self.temp), 1)[:, 0] 
        self.opt = tf.train.AdamOptimizer(learning_rate).minimize(self.loss, global_step=self.gstep)

    def train(self):
        saver = tf.train.Saver()
        start = time.time()
        min_loss = None

        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)

        with tf.Session(config = tf.ConfigProto(gpu_options=gpu_options)) as sess:
            sess.run(tf.global_variables_initializer())

            ckpt = tf.train.get_checkpoint_state(os.path.dirname('models/' + self.model + '/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)

            iteration = self.gstep.eval()
            data = read_batch(
                read_data(self.path, num_steps, num_steps // 2),
                batch_size
            )

            while True:
                batch = next(data)
                
                # for batch in read_batch(read_data(DATA_PATH, vocab)):
                batch_loss, _ = sess.run([self.loss, self.opt], {self.seq: batch})
                
                if (iteration + 1) % skip == 0:
                    print('Iter {}. \n    Loss {}. Time {}'.format(iteration + 1, batch_loss, time.time() - start))

                    self.online_infer(sess)
                    start = time.time()
                    checkpoint_name = 'models/' + self.model

                    if min_loss is None:
                        saver.save(sess, checkpoint_name, iteration)
                    elif batch_loss < min_loss:
                        saver.save(sess, checkpoint_name, iteration)
                        min_loss = batch_loss

                iteration += 1

    def online_infer(self, sess):
        for seed in ["anh"]:
            sentence = [seed]
            state = None

            for _ in range(len_generated):
                batch = [vocab_encode(sentence[-1])]
                feed = {self.seq: batch}

                if state is not None: # for the first decoder step, the state is None
                    for i in range(len(state)):
                        feed.update({self.in_state[i]: state[i]})

                index, state = sess.run([self.sample, self.out_state], feed)
                sentence += [vocab_decode(index)]

            print(''.join(sentence))

In [ ]:
lm = RNN('stories')
lm.create_model()

In [ ]:
lm.train()

Iter 50. 
    Loss 14742.322265625. Time 9.963157176971436
anh                                                                                                                                                                                                                                                                                                            
Iter 100. 
    Loss 14269.9404296875. Time 9.079530477523804
anh                                                                                                                                                                                                                                                                                                            
Iter 150. 
    Loss 14025.5859375. Time 9.028253078460693
anh                                                                                                                                                                                                                   

Iter 750. 
    Loss 11323.00390625. Time 9.007134437561035
anh một một người một một một một một một người một người một một một một một người một một một một một một một một người người một một một một một một người một một một người một một người một một một một một một một người một người một một một một một một người một một một một một một người một một một một người một một một người một một một một một một một một một một người một một một người một một một người một một một người một người một người một một người một một một một người một một một người một một người một một một người một một người một một một một một người người một người một một một một một một một một một một người hụp một một người người một
Iter 800. 
    Loss 12558.7783203125. Time 9.009844064712524
anh bà một không một một một một một một một một một một một một một anh làm một một một người một bà một một anh một bà làm một một một một một một không người làm bà làm làm một bà một một người làm làm một a